## Homework: Vector Search

### Q1. Getting the embeddings model


In [1]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"

embedding_model = SentenceTransformer(model_name)

/home/daniel_tomiwa/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


What is the first value of the resulting vector?

In [2]:
# Encode the sentence
sentence_embedding = embedding_model.encode("I just discovered the course. Can I still join it?")

# Extract the first number
first_digit = sentence_embedding[0]

print(f"The first digit is: {first_digit}")

The first digit is: 0.07822263240814209


In [3]:
import numpy as np

# Save the question vector
np.save('data/question_vector.npy', sentence_embedding)

### Prepare the documents

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'

docs_url = f'{base_url}/{relative_url}?raw=1'

docs_response = requests.get(docs_url)

documents = docs_response.json()

documents[:5]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

In [4]:
# filter the documents for only the course machine learning zoomcamp

def filter_by_course(course: str, initial_list: list) -> dict:

    final_list = []

    # Populate the final list with only the selected course
    for item in initial_list:
        if item["course"] == course:
            final_list.append(item)

    return final_list

filtered_docs = filter_by_course(course="machine-learning-zoomcamp", initial_list=documents)

print(len(filtered_docs))

375


### Q2. Creating the embeddings

In [5]:
import numpy as np

embeddings = []

for document in filtered_docs:
    # Extract some fields from the document
    question = document["question"]
    text = document["text"]
    # Create a single string from the fields
    qa_text = f'{question} {text}'

    # Create an embedding from the string
    emb = embedding_model.encode(qa_text).tolist()
    # Append to the list
    embeddings.append(emb)

X = np.array(embeddings)

What's the shape of X? (X.shape). Include the parantheses.

In [6]:
print(X.shape)

(375, 768)


In [7]:
# Check the cosine similarity between the query vector and the embedding matrix
scores = X.dot(sentence_embedding)

What's the highest score in the results?

In [8]:
print(max(scores))

0.6506574159551753


## Vector search

In [9]:
class VectorSearchEngine():

    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]
    

search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(v_query=sentence_embedding, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'

ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)

df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']

ground_truth = df_ground_truth.to_dict(orient='records')

for item in ground_truth:
    question = item["question"]
    question_vector = embedding_model.encode(question)
    # Add the vector to the item
    item["question_vector"] = question_vector.tolist()

In [10]:
ground_truth[:5]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872',
  'question_vector': [0.09801879525184631,
   -0.003254842245951295,
   0.05209549143910408,
   -0.0077957832254469395,
   0.0867941603064537,
   0.038857102394104004,
   0.04648662358522415,
   0.03894266113638878,
   0.005926491227000952,
   0.01113617978990078,
   -0.02468799240887165,
   -0.04955799877643585,
   0.1013110801577568,
   -0.022801578044891357,
   0.021167999133467674,
   -0.01749049685895443,
   0.04091059789061546,
   -0.020109044387936592,
   -0.011428127065300941,
   -0.04240448400378227,
   -0.01006295159459114,
   0.015109118074178696,
   0.02568870596587658,
   0.018088895827531815,
   -0.027396388351917267,
   0.06961420923471451,
   0.05531587079167366,
   0.02749752812087536,
   -0.03616594150662422,
   -0.021262336522340775,
   0.016303911805152893,
   0.026281289756298065,
   0.038237813860177994,
   0.006220387294888496,
   0.027246458455

In [12]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        
        doc_id = q['document']

        # get the vector representation of the question
        q_vec = embedding_model.encode(q['question'])

        results = search_function(q_vec)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [13]:
evaluate(
    ground_truth=ground_truth,
    search_function=search_engine.search
)

100%|██████████| 1830/1830 [00:45<00:00, 40.42it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

Add the embeddings as a field in the filtered docs

In [14]:
for doc, vector in tqdm(zip(filtered_docs, embeddings)):
    doc['question_text_vector'] = vector

375it [00:00, 220505.26it/s]


Save the ground truth and the filtered_docs documents

In [11]:
import json

with open("data/filtered_docs.json", mode="w") as output_docs:
    json.dump(filtered_docs, output_docs, indent=2)

with open("data/ground_truth.json", mode="w") as ground_output:
    json.dump(ground_truth, ground_output, indent=2)

### Q5. Indexing with Elasticsearch

In [1]:
import json
from tqdm.auto import tqdm
import numpy as np

# Load the filtered docs and the ground truth data
with open("data/filtered_docs.json", mode="r") as input_docs:
    filtered_docs = json.load(input_docs)

with open("data/ground_truth.json", mode="r") as input_docs:
    ground_truth = json.load(input_docs)

# Load the question vector
question_vector = np.load("data/question_vector.npy")

/home/daniel_tomiwa/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Index the documents into elastic search

In [3]:
for doc in tqdm(filtered_docs):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [00:03<00:00, 102.42it/s]


In [4]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [5]:
elastic_search_knn(
    field="question_text_vector",
    vector=question_vector,
    course="machine-learning-zoomcamp"
)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [12]:
def question_vector_knn(q):

    course = "machine-learning-zoomcamp"

    return elastic_search_knn('question_text_vector', q, course)

In [13]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        
        doc_id = q['document']
        # get the vector representation of the question
        q_vec = q['question_vector']

        results = search_function(q_vec)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [14]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 1830/1830 [00:28<00:00, 64.86it/s] 


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}